# What you already know: HOFs and Scala collections

I assume you already know the basics of Scala and higher-order functions, at least as they are used in the Scala Collections library. 

In [10]:
// example of List processing with map, filter, etc. 

List(1,2,3,4)
    .map(_ + 1)
    .filter(_ % 2 == 0)
    .reduce(_ + _)

res9: Int = 6

# Spark: standalone setup

Spark is a distributed processing framework for transforming big data sets using the computational power of a dedicated cluster. But we can use Spark in an standalone mode (i.e. with no cluster at all), for testing or pedagogical purposes. In that case, we just exploit the cores of your local processor.

### Create the Spark session

The Spark session is the entry point to the Spark interpreter. We need it for running Spark programs.

In [11]:
// Create a Spark session in standalone mode

import $ivy.`org.apache.spark::spark-sql:2.4.3` 
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}

val spark: SparkSession = 
    NotebookSparkSession
      .builder()
      .master("local[*]") //frame local, * todos los cores disponibles en el portatil
// puede poner el numero de cores que quieres utilizar: local[2]    
      .getOrCreate()


Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.3/spark-sql_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.12/0.9.1/almond-spark_2.12-0.9.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/2.4.3/spark-parent_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-mapreduce/1.5.5/orc-mapreduce-1.5.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-column/1.10.1/parquet-column-1.10.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/2.4.3/spark-catalyst_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-hadoop/1.10.1/parquet-hadoop-1.10.1.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.8/scala-library-2.12.8.pom
Downloaded https://repo1.maven.org/maven2/org/spark-project/spark/

Downloaded https://repo1.maven.org/maven2/org/codehaus/jackson/jackson-core-asl/1.9.13/jackson-core-asl-1.9.13.pom
Downloaded https://repo1.maven.org/maven2/joda-time/joda-time/2.9.9/joda-time-2.9.9.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/jcl-over-slf4j/1.7.16/jcl-over-slf4j-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-client/2.6.5/hadoop-client-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/com/twitter/chill-java/0.9.3/chill-java-0.9.3.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-scala_2.12/2.6.7.1/jackson-module-scala_2.12-2.6.7.1.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/containers/jersey-container-servlet/2.22.2/jersey-container-servlet-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-shims/1.5.5/orc-shims-1.5.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-memory/0.10.0/arrow-memory-0.10.0.pom
Downloaded https://r

Downloaded https://repo1.maven.org/maven2/org/slf4j/jul-to-slf4j/1.7.16/jul-to-slf4j-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-parser-combinators_2.12/1.1.0/scala-parser-combinators_2.12-1.1.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/curator/curator-recipes/2.6.0/curator-recipes-2.6.0.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/core/jersey-client/2.22.2/jersey-client-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/net/sf/py4j/py4j/0.10.7/py4j-0.10.7.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-format/2.4.0/parquet-format-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-encoding/1.10.1/parquet-encoding-1.10.1.pom
Downloaded https://repo1.maven.org/maven2/com/github/luben/zstd-jni/1.3.2-2/zstd-jni-1.3.2-2.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-annotations/2.6.0/jackson-annotations-2.6.0.pom
Downloaded https

Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-parent/2.7/jackson-parent-2.7.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-parent/1.7.16/slf4j-parent-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/40/commons-parent-40.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-parent/2.6.1/jackson-parent-2.6.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/35/commons-parent-35.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/34/commons-parent-34.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/23/commons-parent-23.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/17/commons-parent-17.pom
Downloaded https://repo1.maven.org/maven2/net/java/jvnet-parent/4/jvnet-parent-4.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/oss-parent/25/oss-parent-25.pom
Downloaded

Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-compress/1.8.1/commons-compress-1.8.1.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/bundles/repackaged/jersey-guava/2.22.2/jersey-guava-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/javax/ws/rs/javax.ws.rs-api/2.0.1/javax.ws.rs-api-2.0.1.pom
Downloaded https://repo1.maven.org/maven2/org/tukaani/xz/1.5/xz-1.5.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/media/jersey-media-jaxb/2.22.2/jersey-media-jaxb-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/curator/curator-framework/2.6.0/curator-framework-2.6.0.pom
Downloaded https://repo1.maven.org/maven2/com/esotericsoftware/kryo-shaded/4.0.2/kryo-shaded-4.0.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-app/2.6.5/hadoop-mapreduce-client-app-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/javax/validation/validation-api/1.1.0.Final/validation-api-1.1.0.Final.pom
D

...

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/02/24 14:10:04 INFO SparkContext: Running Spark version 2.4.3
20/02/24 14:10:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
20/02/24 14:10:05 INFO SparkContext: Submitted application: ddac7178-dd7c-4e82-890a-fe96bc2b9e09
20/02/24 14:10:05 INFO SecurityManager: Changing view acls to: jovyan
20/02/24 14:10:05 INFO SecurityManager: Changing modify acls to: jovyan
20/02/24 14:10:05 INFO SecurityManager: Changing view acls groups to: 
20/02/24 14:10:05 INFO SecurityManager: Changing modify acls groups to: 
20/02/24 14:10:05 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
20/02/24 14:10:06 INFO Utils: Successfully started service 'sparkD

20/02/24 14:10:07 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0-sources.jar at spark://0e8a1150599b:33707/jars/scala-collection-compat_2.12-2.0.0-sources.jar with timestamp 1582553407178
20/02/24 14:10:07 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0.jar at spark://0e8a1150599b:33707/jars/scala-collection-compat_2.12-2.0.0.jar with timestamp 1582553407183
20/02/24 14:10:07 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.2.0/scala-xml_2.12-1.2.0-sources.jar at spark://0e8a1150599b:33707/jars/scala-xml_2.12-1.2.0-sources.jar with timestamp 1582553407187
20/02/24 14:10:07 INFO SparkContext: Added JAR file:/home/jovyan/.cache

20/02/24 14:10:07 INFO BlockManagerMaster: Registered BlockManager BlockManagerId(driver, 0e8a1150599b, 39049, None)
20/02/24 14:10:07 INFO BlockManager: Initialized BlockManager: BlockManagerId(driver, 0e8a1150599b, 39049, None)


Spark UI

import $ivy.$                                   

import $ivy.$                              


import org.apache.spark.sql.{NotebookSparkSession, SparkSession}


spark: SparkSession = org.apache.spark.sql.SparkSession@64fe77c6

### Logging configuration

This is convenient to minimize the amount of info displayed in the terminal.

In [12]:
import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}
Logger.getRootLogger().setLevel(Level.ERROR) // bajar a nivel de error
//DEBUG, INFO,ERROR

import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


### Useful imports

In [13]:
import spark.implicits._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._

import spark.implicits._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._

# Your first Spark program 

The following program is an example of a _Dataset_ program. The Dataset API is one of the languages for distributed data processing that the Spark framework provides. We will omit reference in this notebook to other APIs such as RDDs, DataFrames, etc.

In [15]:
List(1,2,3,4).toDS
    .map(_ + 1)
    .filter(i => i % 2 == 0)
    .reduce(_ + _)
    

reduce at cmd14.sc:4

1 / 1

res14: Int = 6

In principle, this Dataset program does not differ significantly from the Scala collection program. Syntactically, the only difference appears to be the `.toDS` expression: 

In [16]:
List(1,2,3,4).toDS

res15: Dataset[Int] = [value: int]

But, of course, there are several major differences.

# First difference: performance 
//Rendimiento

Let's define a heavy computation:

In [17]:
def heavyComp(ms: Int = 1000)(x: Int) = {
  Thread.sleep(ms)
  x+1
}

defined function heavyComp

and a way to measure execution time:

In [18]:
def run[A](code: => A): A = {
    //=> A : tipo A delayed(demorado), ese argumento no se va a valuar hasta que se utilice
    val start = System.currentTimeMillis()
    val res = code
    println(s"Took ${System.currentTimeMillis() - start}")
    res
}
    
    // code: Unit => A se puede ver el tipo delayed como una funcion 
    // val res = code(())

defined function run

The following Scala Collection program takes some time to execute:

In [19]:
run(List(1,2,3,4).map(heavyComp()).reduce(_ + _))

Took 4003


res18: Int = 14

However, the equivalent Dataset program takes half time (or less time depending on the number of cores of your processor)!

In [23]:
run(List(1,2,3,4).toDS.map(heavyComp()).reduce(_ + _)) //Spark paraliza los procesos en los cores

reduce at cmd22.sc:1

1 / 1

Took 4267


res22: Int = 14

The Dataset program run faster because the Spark framework is designed to take advantage of the parallel and distributed architecture of your computing infrastructure. In our case, it simply exploits the number of cores of your processor.

However, note that using Spark to parallelize your code is overkill. If you are not in a truly distributed setting, you can get along the same benefits more simply using the parallel collections framework of the Scala standard library: 

In [22]:
run(List(1,2,3,4).par.map(heavyComp()).reduce(_ + _)) 
// collecion tiene un metodo para paralelizar el procesamiento sin utilizar spark, si queremos trabajar local y pocos datos

Took 4013


res21: Int = 14

# Second difference: _laziness_
//declaratividad: la transformacio/operacion no se hace hasta que no se collect .

Let's compare this Scala collection transformation:

In [24]:
val result: List[Int] = List(1,2,3,4).map(_ + 1)

result: List[Int] = List(2, 3, 4, 5)

with the following Dataset one:

In [25]:
val program: Dataset[Int] = List(1,2,3,4).toDS.map(_ + 1)
    // solo estamos declarando que es lo que tiene que hacer

program: Dataset[Int] = [value: int]

We obtain no transformation at all! Dataset programs are that: _programs_. We won't find any data in an instance of `Dataset`, just a program or _generator_ of a data set. A `Dataset` declares a number of _transformations_ that will be eventually enacted with specific _actions_. For instance, using `collect`:

In [26]:
val result: Array[Int] = program.collect
    // hasta que no hacemos collect , no tenemos la transformacion hecha como tal

collect at cmd25.sc:1

1 / 1

result: Array[Int] = Array(2, 3, 4, 5)

or `reduce`:

In [27]:
val result2: Int = program.reduce(_ + _)
    //acciones: reduce,collect
    // transformaciones: map, filter (son declarativas)    
    // Dentro de DataSet[] no hay datos, sino instrucciones/programas para generar datos. 

reduce at cmd26.sc:1

1 / 1

result2: Int = 14

Spark Datasets are said to be *lazy*, because we don't inmediately obtain an answer. Rather, we _declaratively_ specify a number of transformations to be applied, and wait until a specific action interprets the transformation program to obtain the desired result. And the same program may be interpreted differently: we may simply want to execute the transformations using `collect`, or may want to perform some calculation using `reduct`. This difference between _transformations_ and _actions_ is reflected very precisely in the [Dataset API](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset).  

### Inspecting the structure of a Spark program

A `Dataset` is a program that it's compiled into a lower-level program before it can be actually executed. The compiler of datasets is called _catalyst_. We can inspect the execution plan that is generated for a particular dataset using `explain`:

In [28]:
List(1,2,3,4).toDS.map(_ + 1).explain
// como pretende ejecutar Spark esa transformacion

== Physical Plan ==
*(1) SerializeFromObject [input[0, int, false] AS value#56]
+- *(1) MapElements ammonite.$sess.cmd27$Helper$$Lambda$4569/970828884@73f34caa, obj#55: int
   +- *(1) DeserializeToObject value#51: int, obj#54: int
      +- LocalTableScan [value#51]


The execution plan of a dataset is in turn compiled into an `RDD`, i.e. a lower-level abstraction program:

In [29]:
//RDD=Resilient Distributed Dataset
//lenguaje de nivel de abstracion baja, lenguaje basico, como explain
// tambien es laziness
//permite ser más resistente ante fallos.

List(1,2,3,4).toDS.map(_ + 1).rdd.toDebugString


res28: String = """(1) MapPartitionsRDD[27] at rdd at cmd28.sc:1 []
 |  MapPartitionsRDD[26] at rdd at cmd28.sc:1 []
 |  MapPartitionsRDD[25] at rdd at cmd28.sc:1 []
 |  ParallelCollectionRDD[24] at rdd at cmd28.sc:1 []"""

### Spark and functional programming

This distinction lies also at the heart of functional programming. On the one hand, there are _programs_ written in a DSL. On the other, there are _interpreters_ that run this program according to different semantics. This is also reflected in the Scala Collections API, particularly, in the notion of _views_. For instance, the following transformation is similarly _lazy_: 

In [30]:
List(1,2,3,4).view.map(heavyComp())

res29: collection.SeqView[Int, Seq[_]] = SeqView(2, 3, 4, 5)

The transformation is only executed when we execute the view using, e.g. `toList`: 

In [31]:
// problema de rendimiento por crear listas intermedias
//List(1,2,3,4).map(_+1).filter(_ %2 ==0)

List(1,2,3,4).view.map(heavyComp()).toList

res30: List[Int] = List(2, 3, 4, 5)

`SeqView`s are *programs*, much in the same way as `Dataset` objects, whereas `toList` is an *action*, equivalent to the `Dataset` actions `collect` and `reduce`.

Similarly, Scala _iterators_ are also good examples of *lazyness*. When we create an iterator from a collection, as in:

In [32]:
val it: Iterator[Int] = List(1,2,3,4).iterator

it: Iterator[Int] = non-empty iterator

and specify a number of transformations:

In [33]:
val it2: Iterator[Int] = 
    List(1,2,3,4).iterator.map(_ + 1).filter(_ % 2 == 0)

it2: Iterator[Int] = non-empty iterator

we don't inmediately obtain those transformations. We are just creating a new iterator that will generate the correspoding data when we ask for it:

In [34]:
it2.toList

res33: List[Int] = List(2, 4)

Actually, there is a close relationship between Spark RDDs (the transformation language in which `Dataset`s are actually translated into), and iterators. 

# Third difference: the execution framework
//La ejecucion.
//Una es secuencial y otra es concurrente, distribuida
// vamos a dividir la Dataset en particiones y vamos crear grafos de tareas

When an action is applied on a `Dataset` program a _job_ is executed by the distributed platform of Spark through a sequence of *stages*; in each stage, the work to be done is performed concurrently by a number of _tasks_. 

The so-called [Spark UI](http://localhost:4040/) allows us to debug the execution process of all the jobs that are submitted for execution through a given Spark session. For instace, the following action launches a job that can be inspected through the Spark UI: 

In [35]:
val ds: Dataset[Int] = List(1,2,3,4).toDS.map(heavyComp())

ds: Dataset[Int] = [value: int]

In [36]:
ds.collect

collect at cmd35.sc:1

1 / 1

res35: Array[Int] = Array(2, 3, 4, 5)

Each bar in the notebook execution corresponds to one stage of the job exectuion; the X/Y label in each bar indicates the number of tasks already executed (X) and the total number of tasks of that stage (Y). 

We can access to the work performed by tasks in each partition through `foreachPartition`:

In [37]:
ds.foreachPartition{ it : Iterator[Int] => 
    println(s"Task output: " + it.toList)
}

foreachPartition at cmd36.sc:1

1 / 1

Task output: List(2, 3, 4, 5)


The number of tasks scheduled for each stage depends on the number of partitions associated to the dataset. When the dataset is first created from a Scala collection, the number of partitions defaults to the number of cores specified when the Spark context was created. 

In [38]:
List(1,2,3,4).toDS.rdd.getNumPartitions

res37: Int = 1

The number of partitions can be set to a specific value using `repartition`: 

In [39]:
List(1,2,3,4,5,6,7,8,9,10,11,12).toDS
    .repartition(12)
    .map(heavyComp(2000))
    .collect

collect at cmd38.sc:3

1 / 1

collect at cmd38.sc:3

12 / 12

res38: Array[Int] = Array(13, 12, 4, 8, 11, 6, 7, 9, 3, 10, 5, 2)

# Shuffling: narrow vs. wide transformations
//cuando hay comunicacion entre particiones se crea nuevo stage

Note that a new stage is created when the dataset is repartitioned. More commonly, new stages are created when so-called _wide_ transformations are interpreted. _Narrow_ transformations are those transformations which are not wide: `map`, `filter`, etc. For instance, this program will execute in one stage: 

In [ ]:
List(1,2,3,4).toDS.map(_ + 1).collect

and the following one as well:

In [ ]:
List(1,2,3,4).toDS.map(_ + 1).filter(_ % 2 == 0).collect

However, this one introduces a new stage:

In [ ]:
// groupBy or some wide transformation

Why? where is the difference? What does a stage means? 

In [ ]:
// TBD

# Dataset API: narrow transformations

# Dataset API: wide transformations